# 투자 분석 AI agent

In [1]:
import os
import warnings
warnings.filterwarnings('ignore')

# 라이브러리 임포트

In [ ]:
from crewai import Agent, Task, Crew, Process, LLM
from crewai.tools import tool
from crewai_tools import SerperDevTool

import yfinance as yf
from datetime import datetime, timedelta
import pandas as pd

from langchain_openai import ChatOpenAI

# 환경 변수 로딩

In [3]:
import os

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")

# 툴 설정

In [ ]:
# 사용자 정의 재무분석 툴 로딩
from stock_analysis import comprehensive_stock_analysis

# 웹 겸색 툴 설정
search_tool = SerperDevTool()

# 

# 에이전트 설정

## LLM 객체 생성

In [5]:
current_time = datetime.now()
llm = LLM(model = "openai/gpt-4o-mini")
invest_llm = LLM(model = "openai/o3-mini-2025-01-31")
# invest_llm = LLM(model="anthropic/claude-3-5-sonnet-20240620")

## 재무 분석가 Agent

In [6]:
# 재무 분석가
financial_analyst = Agent(
    role="Financial Analyst",
    goal="회사의 재무 상태 및 성과 분석",
    backstory="당신은 재무 제표와 비율을 해석하는 데 전문성을 갖춘 노련한 재무 분석가입니다.날짜: {current_time:%Y년 %m월 %d일}",
    tools=[comprehensive_stock_analysis],
    llm=llm,
    max_iter=3,
    allow_delegation=False,
    verbose=True
)

## 시장 분석가

In [7]:
# 시장 분석가
market_analyst = Agent(
    role="Market Analyst",
    goal="회사의 시장 지위 및 업계 동향 분석",
    backstory="당신은 기업/산업 현황 및 경쟁 환경을 전문적으로 분석할 수 있는 숙련된 시장 분석가입니다.날짜: {current_time:%Y년 %m월 %d일}",
    tools=[search_tool],
    llm=llm,
    max_iter=3,
    allow_delegation=False,
    verbose=True
)

## 위험 분석가

In [8]:
# 위험 분석가
risk_analyst = Agent(
    role="Risk Analyst",
    goal="주식과 관련된 잠재적 위험 식별 및 평가",
    backstory="당신은 투자에서 명백한 위험과 숨겨진 위험을 모두 식별하는 예리한 안목을 갖춘 신중한 위험 분석가입니다.날짜: {current_time:%Y년 %m월 %d일}",
    tools=[comprehensive_stock_analysis],
    llm=llm,
    allow_delegation=False,
    verbose=True
)

## 투자 전문가

In [9]:
# 투자 전문가
investment_advisor = Agent(
    role="Investment Advisor",
    goal="전체 분석을 기반으로 한 투자 추천 제공",
    backstory="다양한 분석을 종합하여 전략적 투자 조언을 제공하는 신뢰할 수 있는 투자 자문가입니다.날짜: {current_time:%Y년 %m월 %d일}",
    llm=invest_llm,
    allow_delegation=False,
    verbose=True
)

In [10]:
def get_user_input():
    ticker = input("투자 자문을 구하고 싶은 기업명을 입력해주세요: ")
    return ticker

In [11]:
def create_dynamic_tasks(ticker):
    financial_analysis = Task(
        description=f"""{ticker}에 대한 철저한 재무 분석을 수행합니다. 
        주요 재무 지표에 집중하세요. 
        회사의 재무 건전성 및 성과 추세에 대한 인사이트를 제공합니다.날짜: {current_time:%Y년 %m월 %d일}""",
        agent=financial_analyst,
        expected_output=f"""{ticker}의 재무 상태에 대한 종합적인 분석 보고서. 
        주요 재무 지표, 수익성, 부채 비율 등을 포함하며, 
        회사의 재무 건전성과 성과 동향에 대한 인사이트를 제공해야 합니다."""
    )

    market_analysis = Task(
        description=f"""{ticker}의 시장 위치를 분석합니다. 
        경쟁 우위, 시장 점유율, 업계 동향을 평가하세요. 
        회사의 성장 잠재력과 시장 과제에 대한 인사이트를 제공하세요.날짜: {current_time:%Y년 %m월 %d일}""",
        agent=market_analyst,
        expected_output=f"""{ticker}의 시장 위치에 대한 상세한 분석 보고서. 
        경쟁 우위, 시장 점유율, 산업 동향을 평가하고, 
        회사의 성장 잠재력과 시장 과제에 대한 인사이트를 포함해야 합니다."""
    )

    risk_assessment = Task(
        description=f"""{ticker}에 대한 투자와 관련된 주요 위험을 파악하고 평가합니다. 
        시장 위험, 운영 위험, 재무 위험 및 회사별 위험을 고려하세요. 
        종합적인 위험 프로필을 제공합니다.날짜: {current_time:%Y년 %m월 %d일}""",
        agent=risk_analyst,
        expected_output=f"""{ticker} 투자와 관련된 주요 리스크에 대한 포괄적인 평가 보고서. 
        시장 리스크, 운영 리스크, 재무 리스크, 회사 특정 리스크를 고려하여 
        종합적인 리스크 분석 결과를 제시해야 합니다."""
    )

    investment_recommendation = Task(
        description=f"""{ticker}의 재무 분석, 시장 분석, 위험 평가를 바탕으로 종합적인 투자 추천을 제공합니다. 
        주식의 잠재 수익률, 위험 및 다양한 유형의 투자자에 대한 적합성을 고려하여 마크다운으로 작성해 주세요.
        한글로 작성하세요.날짜: {current_time:%Y년 %m월 %d일}""",
        agent=investment_advisor,
        expected_output=f"""
        1. 제목 및 기본 정보
           - 회사명, 티커, 현재 주가, 목표주가, 투자의견 등
        
        2. 요약(Executive Summary)
           - 핵심 투자 포인트와 주요 재무 지표를 간단히 정리
        
        3. 기업 개요
           - 회사의 주요 사업 영역, 연혁, 시장 점유율 등
        
        4. 산업 및 시장 분석
           - 해당 기업이 속한 산업의 트렌드와 전망
        
        5. 재무 분석
           - 매출, 영업이익, 순이익 등 주요 재무지표 분석
           - 수익성, 성장성, 안정성 지표 분석
        
        6. 밸류에이션
           - P/E, P/B, ROE 등 주요 밸류에이션 지표 분석
           - 경쟁사 대비 상대 밸류에이션
        
        7. 투자 의견 및 목표주가
           - 투자의견 제시 및 근거 설명
           - 목표주가 산정 방법과 근거
        
        8. 투자 위험 요인
           - 잠재적인 리스크 요인들을 나열
        
        9. 재무제표 요약
           - 최근 몇 년간의 요약 손익계산서, 재무상태표, 현금흐름표

        """,
        output_file=r"./investment_advisor.md" 
    )

    return [financial_analysis, market_analysis, risk_assessment, investment_recommendation]

# Crew 객체 생성

In [12]:
# ticker = get_user_input()
ticker = "AAPL"
tasks = create_dynamic_tasks(ticker)

crew = Crew(
    agents=[financial_analyst, market_analyst, risk_analyst, investment_advisor],
    tasks=tasks,
    process=Process.sequential,
    output_log_file="logs/investment_advisor_run.json", 
    verbose=False
)


# crew 실행

In [13]:
result = crew.kickoff()

# Agent: Financial Analyst
## Task: AAPL에 대한 철저한 재무 분석을 수행합니다. 
        주요 재무 지표에 집중하세요. 
        회사의 재무 건전성 및 성과 추세에 대한 인사이트를 제공합니다.날짜: 2025년 06월 23일


# Agent: Financial Analyst
## Thought: Action: Updated Comprehensive Stock Analysis
## Using tool: Updated Comprehensive Stock Analysis
## Tool Input: 
"{\"ticker\": \"AAPL\"}"
## Tool Output: 
{'현재 주가': {'현재 주가': np.float64(201.0), '기준 시간': '2025-06-20 00:00:00'}, '연간 데이터': {'매출': '391,035,000,000', '매출원가': '210,352,000,000', '매출총이익': '180,683,000,000', '영업이익': '123,216,000,000', '순이익': '93,736,000,000', 'EBITDA': '134,661,000,000', '매출총이익률': '46.21%', '영업이익률': '31.51%', '순이익률': '23.97%', '매출 성장률': '2.02%', '순이익 성장률': '-3.36%', '희석주당순이익(EPS)': '$6.08', '부채비율': '84.40%'}, '분기 데이터': {'매출': '95,359,000,000', '순이익': '24,780,000,000', '매출 성장률(QoQ)': '-23.28%', '순이익 성장률(QoQ)': '-31.79%'}, '연간 재무제표 요약': {'2024-09-30': {'총수익': '391,035,000,000', '영업이익': '123,216,000,000', '순이익': '93,736,000,000', 'EBITDA': '134,661,000,000', 'EPS(희석)': '$6.08

In [14]:
from IPython.display import display, Markdown
display(Markdown(result.raw))

# AAPL 투자 분석 보고서

## 1. 제목 및 기본 정보
- **회사명:** Apple Inc.
- **티커:** AAPL
- **현재 주가:** $201.00
- **목표주가:** $220.00 (재무적 안정성과 성장성, 경쟁력, 그리고 최근 분기 실적 감소를 모두 감안한 산출)
- **투자의견:** 중립 – 혁신과 서비스 부문의 성장 가능성은 긍정적이나, 최근 분기의 매출 및 순이익 하락, 높은 부채비율 등 리스크 요인을 고려하여 신중한 접근이 요구됨

## 2. 요약 (Executive Summary)
- **핵심 투자 포인트:** 
  - Apple은 강력한 브랜드 파워와 프리미엄 제품 라인을 통해 높은 수익성을 유지하고 있음.
  - 서비스 부문의 성장과 기술 혁신이 미래 성장을 견인할 전망이나, 최근 분기 실적 하락 및 부채 비율(84.40%) 증가가 우려됨.
  - 경쟁 심화, 규제 강화, 공급망 불안 등 외부 리스크에 신중히 대응할 필요가 있음.
- **주요 재무 지표:** 
  - 총 매출: $391,035,000,000, 매출 총이익률 46.21%, 영업이익률 31.51%, 순이익률 23.97%
  - 연도별 성장률은 다소 감소 추세를 보임 (매출 성장률 2.02%, 순이익 성장률 -3.36%)

## 3. 기업 개요
- **사업 영역:** 
  - Apple은 스마트폰(iPhone), 태블릿(iPad), 컴퓨터(Mac), 스마트워치(Apple Watch) 및 기타 액세서리와 함께 서비스(음악, 클라우드 서비스, 앱스토어 등)를 제공
- **연혁 및 시장 점유율:** 
  - 오랜 역사와 혁신을 바탕으로 전 세계 프리미엄 전자제품 시장에서 독보적인 시장 점유율을 보유
  - 프리미엄 스마트폰 시장에서 약 50% 이상의 점유율을 기록하며 강력한 브랜드 충성도를 유지

## 4. 산업 및 시장 분석
- **산업 트렌드:**
  - 인공지능(AI) 및 AR/VR 기술의 통합으로 제품 다각화 및 기능 고도화가 진행 중
  - 지속 가능한 경영과 환경 친화적 제품에 대한 수요가 증가
  - 글로벌 경제 불확실성과 인플레이션, 소비 패턴 변화 등 외부 요인이 시장 전반에 영향을 미침
- **경쟁 환경:**
  - 고가 프리미엄 시장에서의 강력한 입지를 유지하고 있으나, 가격 경쟁력을 앞세운 Android 기반 경쟁사들이 지속적으로 시장 점유율을 확대하려는 움직임 있음

## 5. 재무 분석
- **주요 재무 지표 (2024년 기준):**
  - 총 매출: $391,035,000,000
  - 영업이익: $123,216,000,000
  - 순이익: $93,736,000,000
  - EBITDA: $134,661,000,000
- **수익성:**
  - 매출 총이익률 46.21%, 영업이익률 31.51%, 순이익률 23.97% – 높은 수익성을 유지
- **성장성:**
  - 연간 매출 성장률은 2.02%로 다소 둔화되었으며, 최근 분기에서는 매출 및 순이익이 각각 -23.28%, -31.79%로 큰 하락을 보임
- **안정성:**
  - 부채비율 84.40%는 높은 편으로, 과중한 부채가 향후 재무 안정성에 부담 요인이 될 수 있음

## 6. 밸류에이션
- **주요 밸류에이션 지표:**
  - P/E, P/B, ROE 등 지표는 Apple의 높은 수익성과 시장 신뢰도를 반영하나, 최근 실적 둔화로 인한 밸류에이션 프리미엄이 일부 조정될 가능성이 있음
- **경쟁사 대비 상대 밸류에이션:**
  - 동종 업계 경쟁사에 비해 프리미엄을 유지하고 있어, 투자자들은 성장성과 브랜드 가치에 프리미엄을 지불하고 있음
  - 다만, 낮은 성장률과 높은 부채비율은 미래 밸류에이션 조정의 리스크로 작용할 수 있음

## 7. 투자 의견 및 목표주가
- **투자의견:** 
  - 투자자 유형에 따라 상이한 접근이 필요함  
    - **위험 선호 투자자:** 단기 변동성 및 리스크를 감수하고 장기적 혁신과 서비스 성장에 베팅 가능
    - **보수적 투자자:** 최근 분기 실적 둔화와 부채 리스크를 고려할 필요가 있으며, 신중한 관망 추천
  - 전반적으로 Apple의 강력한 브랜드와 서비스 성장 잠재력을 고려하더라도, 최근 실적 하락과 높은 부채 비율은 투자 결정 시 중요한 고려대상이 됨
- **목표주가 산정 근거:** 
  - 현재 주가 $201.00에서 현재 수익성과 프리미엄 밸류에이션, 그리고 개선 가능성이 있는 서비스 부문 상승 효과를 감안하여 $220.00로 설정  
  - 목표주가는 재무 안정성, 성장 전망, 시장 경쟁력 및 외부 경제 요인을 종합적으로 고려한 결과임

## 8. 투자 위험 요인
- **시장 위험:**
  - 글로벌 경제 불확실성, 환율 변동 및 인플레이션 압력 등
  - 경쟁 심화로 인한 프리미엄 시장 내 점유율 변동 가능성
- **운영 위험:**
  - 복잡한 공급망 의존으로 인한 공급 차질 및 지정학적 리스크
  - 혁신 주기 부진 시 제품 경쟁력 저하 가능성
- **재무 위험:**
  - 부채비율 84.40%로 인한 재무적 부담 및 이자 비용 상승 우려
  - 최근 분기 매출과 순이익 하락이 지속될 경우 재무 건전성에 악영향 미칠 가능성
- **규제 및 법률 위험:**
  - 전 세계적으로 강화되고 있는 규제 및 반독점, 개인정보 보호 관련 이슈

## 9. 재무제표 요약
- **2024년 (연간):**
  - 총 수익: $391,035,000,000
  - 영업이익: $123,216,000,000
  - 순이익: $93,736,000,000
  - 희석주당순이익 (EPS): $6.08
- **2023년 (연간):**
  - 총 수익: $383,285,000,000
  - 영업이익: $114,301,000,000
  - 순이익: $96,995,000,000
  - 희석주당순이익 (EPS): $6.13
- **2025년 1분기 (분기):**
  - 총 수익: $95,359,000,000
  - 영업이익: $29,589,000,000
  - 순이익: $24,780,000,000
  - 희석주당순이익 (EPS): $1.65

---

종합하면, Apple Inc. (AAPL)는 강력한 브랜드, 높은 수익성, 혁신적인 기술력과 서비스 부문 성장 가능성을 보유하고 있으나, 최근 분기 실적의 급격한 하락과 높은 부채비율이 투자에 대한 주의를 요합니다. 투자자들은 이러한 리스크를 충분히 고려한 후, 개인의 위험 선호도에 따라 장기적 관점에서 접근하거나 단기 시장 변동에 대비할 필요가 있습니다.  
따라서, 중립적인 투자의견과 함께 신중한 관망 혹은 상황에 따라 단계적인 매수 전략을 추천합니다.